In [1]:
from pyspark.sql import SparkSession

In [2]:
# 스파크 세션 생성
spark = SparkSession.builder.master("local").appName("learn-sql").getOrCreate()

21/12/14 15:47:40 WARN Utils: Your hostname, 6miniui-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.200.112 instead (on interface en0)
21/12/14 15:47:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/14 15:47:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 주식 데이터 생성
stocks = [
    ('Google', 'GOOGL', 'USA', 2984, 'USD'), 
    ('Netflix', 'NFLX', 'USA', 645, 'USD'),
    ('Amazon', 'AMZN', 'USA', 3518, 'USD'),
    ('Tesla', 'TSLA', 'USA', 1222, 'USD'),
    ('Tencent', '0700', 'Hong Kong', 483, 'HKD'),
    ('Toyota', '7203', 'Japan', 2006, 'JPY'),
    ('Samsung', '005930', 'Korea', 70600, 'KRW'),
    ('Kakao', '035720', 'Korea', 125000, 'KRW'),
]

In [4]:
# 스키마 생성
# 컬럼의 이름만 입력하고 데이터 타입은 정하지 않는다.
stockSchema = ["name", "ticker", "country", "price", "currency"]

In [5]:
# 데이터 프레임 생성
df = spark.createDataFrame(data=stocks, schema=stockSchema)

In [8]:
# 데이터 타입 확인
df.dtypes

[('name', 'string'),
 ('ticker', 'string'),
 ('country', 'string'),
 ('price', 'bigint'),
 ('currency', 'string')]

In [38]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- ticker: string (nullable = true)
 |-- country: string (nullable = true)
 |-- price: long (nullable = true)
 |-- currency: string (nullable = true)



In [39]:
# 데이터 프레임 확인
df.show()

+-------+------+---------+------+--------+
|   name|ticker|  country| price|currency|
+-------+------+---------+------+--------+
| Google| GOOGL|      USA|  2984|     USD|
|Netflix|  NFLX|      USA|   645|     USD|
| Amazon|  AMZN|      USA|  3518|     USD|
|  Tesla|  TSLA|      USA|  1222|     USD|
|Tencent|  0700|Hong Kong|   483|     HKD|
| Toyota|  7203|    Japan|  2006|     JPY|
|Samsung|005930|    Korea| 70600|     KRW|
|  Kakao|035720|    Korea|125000|     KRW|
+-------+------+---------+------+--------+



In [40]:
df.select('*').collect()

[Row(name='Google', ticker='GOOGL', country='USA', price=2984, currency='USD'),
 Row(name='Netflix', ticker='NFLX', country='USA', price=645, currency='USD'),
 Row(name='Amazon', ticker='AMZN', country='USA', price=3518, currency='USD'),
 Row(name='Tesla', ticker='TSLA', country='USA', price=1222, currency='USD'),
 Row(name='Tencent', ticker='0700', country='Hong Kong', price=483, currency='HKD'),
 Row(name='Toyota', ticker='7203', country='Japan', price=2006, currency='JPY'),
 Row(name='Samsung', ticker='005930', country='Korea', price=70600, currency='KRW'),
 Row(name='Kakao', ticker='035720', country='Korea', price=125000, currency='KRW')]

In [42]:
df.select('name', 'price').collect()

[Row(name='Google', price=2984),
 Row(name='Netflix', price=645),
 Row(name='Amazon', price=3518),
 Row(name='Tesla', price=1222),
 Row(name='Tencent', price=483),
 Row(name='Toyota', price=2006),
 Row(name='Samsung', price=70600),
 Row(name='Kakao', price=125000)]

In [44]:
df.select(df.name, (df.price + 10000).alias('price')).collect()

[Row(name='Google', price=12984),
 Row(name='Netflix', price=10645),
 Row(name='Amazon', price=13518),
 Row(name='Tesla', price=11222),
 Row(name='Tencent', price=10483),
 Row(name='Toyota', price=12006),
 Row(name='Samsung', price=80600),
 Row(name='Kakao', price=135000)]

In [45]:
df.agg({'price': 'max'}).collect() # 딕셔너리 형태로 컬럼의 max를 가져온다.

[Row(max(price)=125000)]

In [47]:
# 파이스파크의 함수를 이용하여 정의할 수도 있다.
from pyspark.sql import functions as F
df.agg(F.min(df.price)).collect()

[Row(min(price)=483)]

In [10]:
# 스파크 SQL
# TempView에 등록을 하여야 사용할 수 있다.
df.createOrReplaceTempView("stocks")

In [11]:
spark.sql("SELECT nameFROM stocks").show()

+-------+
|   name|
+-------+
| Google|
|Netflix|
| Amazon|
|  Tesla|
|Tencent|
| Toyota|
|Samsung|
|  Kakao|
+-------+



In [12]:
spark.sql("SELECT name, price FROM stocks").show()

+-------+------+
|   name| price|
+-------+------+
| Google|  2984|
|Netflix|   645|
| Amazon|  3518|
|  Tesla|  1222|
|Tencent|   483|
| Toyota|  2006|
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [13]:
spark.sql("SELECT name, price FROM stocks WHERE country = 'Korea'").show()

+-------+------+
|   name| price|
+-------+------+
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [14]:
spark.sql("SELECT name, price FROM stocks WHERE price > 2000").show()

+-------+------+
|   name| price|
+-------+------+
| Google|  2984|
| Amazon|  3518|
| Toyota|  2006|
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [15]:
spark.sql("SELECT name, price FROM stocks WHERE price > 2000 and country = 'USA'").show()

+------+-----+
|  name|price|
+------+-----+
|Google| 2984|
|Amazon| 3518|
+------+-----+



In [17]:
spark.sql("SELECT name, price FROM stocks WHERE country LIKE 'U%'").show() # U로 시작

+-------+-----+
|   name|price|
+-------+-----+
| Google| 2984|
|Netflix|  645|
| Amazon| 3518|
|  Tesla| 1222|
+-------+-----+



In [18]:
spark.sql("SELECT name, price FROM stocks WHERE country LIKE 'U%' AND name NOT LIKE '%e%'").show() # U로 시작하는 컨트리, e를 가진 회사명을 제외

+------+-----+
|  name|price|
+------+-----+
|Amazon| 3518|
+------+-----+



In [16]:
spark.sql("SELECT name, price FROM stocks WHERE price BETWEEN 1000 AND 10000").show() # 1000에서 10000 사이

+------+-----+
|  name|price|
+------+-----+
|Google| 2984|
|Amazon| 3518|
| Tesla| 1222|
|Toyota| 2006|
+------+-----+



In [19]:
spark.sql("SELECT name, price FROM stocks WHERE country='USA'").show()

+-------+-----+
|   name|price|
+-------+-----+
| Google| 2984|
|Netflix|  645|
| Amazon| 3518|
|  Tesla| 1222|
+-------+-----+



In [20]:
spark.sql("SELECT name, price, currency FROM stocks \
WHERE currency = 'USD' AND \
price > (SELECT price FROM stocks WHERE NAME = 'Tesla')").show()

+------+-----+--------+
|  name|price|currency|
+------+-----+--------+
|Google| 2984|     USD|
|Amazon| 3518|     USD|
+------+-----+--------+



In [21]:
spark.sql("SELECT name, price FROM stocks ORDER BY price ASC").show()

+-------+------+
|   name| price|
+-------+------+
|Tencent|   483|
|Netflix|   645|
|  Tesla|  1222|
| Toyota|  2006|
| Google|  2984|
| Amazon|  3518|
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [22]:
spark.sql("SELECT name, price FROM stocks ORDER BY price DESC").show()

+-------+------+
|   name| price|
+-------+------+
|  Kakao|125000|
|Samsung| 70600|
| Amazon|  3518|
| Google|  2984|
| Toyota|  2006|
|  Tesla|  1222|
|Netflix|   645|
|Tencent|   483|
+-------+------+



In [23]:
spark.sql("SELECT name, price FROM stocks ORDER BY LENGTH(name)").show()

+-------+------+
|   name| price|
+-------+------+
|  Tesla|  1222|
|  Kakao|125000|
| Amazon|  3518|
| Toyota|  2006|
| Google|  2984|
|Netflix|   645|
|Samsung| 70600|
|Tencent|   483|
+-------+------+



In [24]:
spark.sql("SELECT sum(price) FROM stocks WHERE country = 'Korea'").show()

+----------+
|sum(price)|
+----------+
|    195600|
+----------+



In [25]:
spark.sql("SELECT mean(price) FROM stocks WHERE country = 'Korea'").show()

+-----------+
|mean(price)|
+-----------+
|    97800.0|
+-----------+



In [26]:
spark.sql("SELECT count(price) FROM stocks WHERE country = 'Korea'").show()

+------------+
|count(price)|
+------------+
|           2|
+------------+



In [27]:
spark.sql("SELECT count(price) FROM stocks WHERE country IN ('Korea', 'USA')").show()

+------------+
|count(price)|
+------------+
|           6|
+------------+



In [16]:
# 여러개의 데이터를 다루는 SQL 예제
earnings = [
    ('Google', 27.99, 'USD'), 
    ('Netflix', 2.56, 'USD'),
    ('Amazon', 6.12, 'USD'),
    ('Tesla', 1.86, 'USD'),
    ('Tencent', 11.01, 'HKD'),
    ('Toyota', 224.82, 'JPY'),
    ('Samsung', 1780., 'KRW'),
    ('Kakao', 705., 'KRW')
]

In [17]:
from pyspark.sql.types import StringType, FloatType, StructType, StructField

In [18]:
# 직접 스키마 타입 설정
earningsSchema = StructType([
    StructField("name", StringType(), True),
    StructField("eps", FloatType(), True),
    StructField("currency", StringType(), True),
])

In [20]:
# 데이터 프레임 생성
earningsDF = spark.createDataFrame(data=earnings, schema=earningsSchema)

In [32]:
earningsDF.dtypes

[('name', 'string'), ('eps', 'float'), ('currency', 'string')]

In [33]:
earningsDF.createOrReplaceTempView("earnings")

In [34]:
earningsDF.select("*").show()

+-------+------+--------+
|   name|   eps|currency|
+-------+------+--------+
| Google| 27.99|     USD|
|Netflix|  2.56|     USD|
| Amazon|  6.12|     USD|
|  Tesla|  1.86|     USD|
|Tencent| 11.01|     HKD|
| Toyota|224.82|     JPY|
|Samsung|1780.0|     KRW|
|  Kakao| 705.0|     KRW|
+-------+------+--------+



In [35]:
spark.sql("SELECT * FROM stocks JOIN earnings ON stocks.name = earnings.name").show()

+-------+------+---------+------+--------+-------+------+--------+
|   name|ticker|  country| price|currency|   name|   eps|currency|
+-------+------+---------+------+--------+-------+------+--------+
| Amazon|  AMZN|      USA|  3518|     USD| Amazon|  6.12|     USD|
| Google| GOOGL|      USA|  2984|     USD| Google| 27.99|     USD|
|  Kakao|035720|    Korea|125000|     KRW|  Kakao| 705.0|     KRW|
|Netflix|  NFLX|      USA|   645|     USD|Netflix|  2.56|     USD|
|Samsung|005930|    Korea| 70600|     KRW|Samsung|1780.0|     KRW|
|Tencent|  0700|Hong Kong|   483|     HKD|Tencent| 11.01|     HKD|
|  Tesla|  TSLA|      USA|  1222|     USD|  Tesla|  1.86|     USD|
| Toyota|  7203|    Japan|  2006|     JPY| Toyota|224.82|     JPY|
+-------+------+---------+------+--------+-------+------+--------+



In [36]:
# PER: Price / EPS 
spark.sql("SELECT stocks.name, (stocks.price / earnings.eps) FROM stocks JOIN earnings ON stocks.name = earnings.name").show()

+-------+------------------+
|   name|     (price / eps)|
+-------+------------------+
| Amazon| 574.8366120563447|
| Google| 106.6095042658442|
|  Kakao| 177.3049645390071|
|Netflix| 251.9531306315913|
|Samsung|39.662921348314605|
|Tencent| 43.86920889728746|
|  Tesla|  656.989242258975|
| Toyota| 8.922693419839167|
+-------+------------------+



In [6]:
df.groupBy().avg().collect()

[Row(avg(price)=25807.25)]

In [7]:
df.show()

+-------+------+---------+------+--------+
|   name|ticker|  country| price|currency|
+-------+------+---------+------+--------+
| Google| GOOGL|      USA|  2984|     USD|
|Netflix|  NFLX|      USA|   645|     USD|
| Amazon|  AMZN|      USA|  3518|     USD|
|  Tesla|  TSLA|      USA|  1222|     USD|
|Tencent|  0700|Hong Kong|   483|     HKD|
| Toyota|  7203|    Japan|  2006|     JPY|
|Samsung|005930|    Korea| 70600|     KRW|
|  Kakao|035720|    Korea|125000|     KRW|
+-------+------+---------+------+--------+



In [10]:
df.groupBy('currency').avg().collect()

[Row(currency='KRW', avg(price)=97800.0),
 Row(currency='JPY', avg(price)=2006.0),
 Row(currency='HKD', avg(price)=483.0),
 Row(currency='USD', avg(price)=2092.25)]

In [11]:
df.groupBy('currency').agg({'price': 'mean'}).collect()

[Row(currency='KRW', avg(price)=97800.0),
 Row(currency='JPY', avg(price)=2006.0),
 Row(currency='HKD', avg(price)=483.0),
 Row(currency='USD', avg(price)=2092.25)]

In [12]:
df.groupBy(df.currency).avg().collect()

[Row(currency='KRW', avg(price)=97800.0),
 Row(currency='JPY', avg(price)=2006.0),
 Row(currency='HKD', avg(price)=483.0),
 Row(currency='USD', avg(price)=2092.25)]

In [14]:
df.groupBy([df.currency, df.price]).count().collect()

[Row(currency='USD', price=1222, count=1),
 Row(currency='USD', price=3518, count=1),
 Row(currency='HKD', price=483, count=1),
 Row(currency='USD', price=645, count=1),
 Row(currency='KRW', price=70600, count=1),
 Row(currency='JPY', price=2006, count=1),
 Row(currency='USD', price=2984, count=1),
 Row(currency='KRW', price=125000, count=1)]

In [25]:

df.join(earningsDF, 'name').select(df.name, earningsDF.eps).collect()

[Row(name='Amazon', eps=6.119999885559082),
 Row(name='Google', eps=27.989999771118164),
 Row(name='Kakao', eps=705.0),
 Row(name='Netflix', eps=2.559999942779541),
 Row(name='Samsung', eps=1780.0),
 Row(name='Tencent', eps=11.010000228881836),
 Row(name='Tesla', eps=1.8600000143051147),
 Row(name='Toyota', eps=224.82000732421875)]

In [26]:
df.show()

+-------+------+---------+------+--------+
|   name|ticker|  country| price|currency|
+-------+------+---------+------+--------+
| Google| GOOGL|      USA|  2984|     USD|
|Netflix|  NFLX|      USA|   645|     USD|
| Amazon|  AMZN|      USA|  3518|     USD|
|  Tesla|  TSLA|      USA|  1222|     USD|
|Tencent|  0700|Hong Kong|   483|     HKD|
| Toyota|  7203|    Japan|  2006|     JPY|
|Samsung|005930|    Korea| 70600|     KRW|
|  Kakao|035720|    Korea|125000|     KRW|
+-------+------+---------+------+--------+



In [27]:
usa_df = df.select('name', 'country', 'price').where('country == "USA"').orderBy('price')
usa_df.show()

+-------+-------+-----+
|   name|country|price|
+-------+-------+-----+
|Netflix|    USA|  645|
|  Tesla|    USA| 1222|
| Google|    USA| 2984|
| Amazon|    USA| 3518|
+-------+-------+-----+



In [28]:
df.groupBy('currency').max('price').show()

+--------+----------+
|currency|max(price)|
+--------+----------+
|     KRW|    125000|
|     JPY|      2006|
|     HKD|       483|
|     USD|      3518|
+--------+----------+



In [29]:
from pyspark.sql.functions import avg, count

df.groupBy('currency').agg(avg('price')).show()

+--------+----------+
|currency|avg(price)|
+--------+----------+
|     KRW|   97800.0|
|     JPY|    2006.0|
|     HKD|     483.0|
|     USD|   2092.25|
+--------+----------+



In [30]:
df.groupBy('currency').agg(count('price')).show()

+--------+------------+
|currency|count(price)|
+--------+------------+
|     KRW|           2|
|     JPY|           1|
|     HKD|           1|
|     USD|           4|
+--------+------------+

